# Calculate contributing area, recharge temperature and heat flux of thermal springs

This notebook uses estimates of groundwater recharge and recorde average discharge of springs to calculate the contributing area of each spring, ie the area that contributes groundwater recharge to each spring. It then draws a circle of the size of the contributing area around each spring and records the minimum and maximum surface elevation in this circle using digital elevation data. This can then be used to estimate the minimum and maximum infiltration temperature of the water that feeds the thermal springs. THis is useful for calculating the heat flow of each spring.

**Input:** ``data/data/thermal_springs_alps_with_geospatial_data.csv`` -> spring dataset with the results of the geospatial data analysis obtained with the ``GIS_analysis_spring_data.ipynb`` notebook.

**Output:** input filename with ``_with_HF_estimates.csv`` appended to it -> contains additional columns for the estimated recharge temperature, the contributing area and the spring heat flux 

In [1]:
import math
import itertools
import string
import chardet

import numpy as np
import pandas as pd
import geopandas as gp
import scipy.stats

import shapely.geometry as gm
import matplotlib.pyplot as pl

#import lib.pyGISlib as pyGISlib

import lib.various_functions as vf

## Parameters

In [2]:
# define locations of spring and raster files. change this to the locations on your own machine or server:
spring_data_file = 'data/thermal_springs_alps_with_geospatial_data.csv'

# shapefile with outline of the Alps
fnb = 'GIS_data/outline_alps_final.shp'

# 
g = 9.81

# specific heat capacity J kg-1 K-1
c = 4181.3

# fluid density
rho = 1000.0

#
degree_symbol = chr(176)

year = 365.25 * 24 * 3600.

# average geothermal gradient
avg_thermal_gradient = 26e-3

# background heat flow density
background_HF = 76e-3

In [3]:
# area of the alps outline shapefile used in GIS_analysis_spring_data.ipynb (m2)
#area_alps = 217759312854.528

# read shapefile with bnd Alps:
dgb = gp.read_file(fnb)

# convert to UTM to calculate area. UTM 32N = epsg: 32632
dgbu = dgb.to_crs({'init': 'epsg:32632'})
area_alps = dgbu.geometry.area[0]

print('used area of alps of %0.2e km2' % (area_alps / 1e6))

used area of alps of 2.02e+05 km2


## Load the thermal springs csv file

In [4]:
def find_encoding(fname):
    r_file = open(fname, 'rb').read()
    result = chardet.detect(r_file)
    charenc = result['encoding']
    return charenc


encoding = find_encoding(spring_data_file)
print(encoding)

UTF-8-SIG


In [5]:
df = pd.read_csv(spring_data_file, encoding=encoding)
#print('columns in csv file: ', df.columns.tolist())

## Get rid of springs without coordinates and convert columns to numeric format:

In [6]:
# make all data numeric
for col in df.columns:
    if 'spring' not in col and 'type' not in col:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else:
        df[col] = df[col]

print(len(df))

394


## Convert to geopandas file:
(code found here https://gis.stackexchange.com/questions/114066/handling-kml-csv-with-geopandas-drivererror-unsupported-driver-ucsv)

In [7]:
df['geometry'] = df.apply(lambda z: gm.Point(z['long'], z['lat']), axis=1)
dg = gp.GeoDataFrame(df)

In [8]:
## Some stats

In [9]:
print('temperature = ', df['temperature'].describe())
print('discharge = ', df['flow_rate'].describe())

ind = df['flow_rate'] == 0
print(df.loc[ind])

temperature =  count    364.000000
mean      22.286951
std       14.662780
min        3.400000
25%       10.450000
50%       18.050000
75%       31.500000
max       70.500000
Name: temperature, dtype: float64
discharge =  count    241.000000
mean       0.011079
std        0.033609
min        0.000001
25%        0.000400
50%        0.001700
75%        0.006450
max        0.305000
Name: flow_rate, dtype: float64
Empty DataFrame
Columns: [spring number, spring location, spring name, long, lat, type, well_depth, flow_rate_mean, flow_rate_min, flow_rate_max, temp_mean, temp_min, temp_max, reference, reference_DOI, reference_link, EC, pH, sample_temperature, TDS, TDS_min, TDS_max, Na, Ca, Mg, K, NH4, Cl, F, SO4, HCO3, CO3, NO3, Si, Li, SiO2, reference_hydrochemistry, doi_hydrochemistry, reference_link_hydrochemistry, delta_18O, delta_2H, 3H, delta_14C, delta_13C, 4He, 3He, reference_isotope_data, DOI_isotope_data, reference_link_isotope_data, temperature, flow_rate, log_flow_rate, log_temper

## Calculate the contributing area for each spring 

by dividing discharge (m3/sec) by recharge (m/s):

In [10]:
dg['contributing_area'] = dg['flow_rate_mean'] / (dg['recharge'] /year)
dg['contributing_area'].describe()

count    2.250000e+02
mean     6.129354e+05
std      1.871053e+06
min      1.468059e+02
25%      1.851829e+04
50%      7.262816e+04
75%      3.877195e+05
max      1.590481e+07
Name: contributing_area, dtype: float64

## Get the min., best and max  recharge temperature

In [11]:
dg['recharge_temp_max'] = dg['surface_temp']
dg['recharge_temp_min'] = dg['min_surface_temp_ws']
#dg['recharge_temp_min'] = dg['min_surface_temp_ws']

dg['recharge_temp_best'] = (dg['surface_temp'] + dg['min_surface_temp_ws']) / 2.0

freezing = dg['recharge_temp_min'] < 0
dg.loc[freezing, 'recharge_temp_min'] = 0.0

freezing = dg['recharge_temp_best'] < 0
dg.loc[freezing, 'recharge_temp_best'] = 0.0

## Get the min., best and max recharge elevation

In [12]:
dg['rch_elev_max'] = dg['max_elevation_ws']
dg['rch_elev_min'] = dg['elevation']
dg['rch_elev_best'] = (dg['max_elevation_ws'] + dg['elevation']) / 2.0

## Calculate max fluid temp based on SiO2

In [13]:
ind_ok = dg['SiO2'] > 5.0
circ_temps = vf.SI_geothermometers(dg.loc[ind_ok, 'SiO2'].values)
dg.loc[ind_ok, 'circulation_temp_best'] = circ_temps[0]
dg.loc[ind_ok, 'circulation_temp_min'] = circ_temps[1]
dg.loc[ind_ok, 'circulation_temp_max'] = circ_temps[2]

print(dg['circulation_temp_best'].describe(), 
      dg['circulation_temp_min'].describe(), 
      dg['circulation_temp_max'].describe())

count    133.000000
mean      64.456307
std       28.875454
min       14.754632
25%       40.378492
50%       60.738275
75%       88.143429
max      129.538405
Name: circulation_temp_best, dtype: float64 count    133.000000
mean      52.316183
std       29.621315
min        0.925394
25%       27.710826
50%       48.551185
75%       76.493728
max      119.471787
Name: circulation_temp_min, dtype: float64 count    133.000000
mean      72.758243
std       25.744020
min       30.031940
25%       51.418549
50%       68.951655
75%       92.809039
max      134.184550
Name: circulation_temp_max, dtype: float64


## Calculate circulation depth

In [14]:
dg['circ_depth_best'] = (dg['circulation_temp_best'] - dg['surface_temp']) / avg_thermal_gradient
dg['circ_depth_min'] = (dg['circulation_temp_min'] - dg['surface_temp']) / avg_thermal_gradient
dg['circ_depth_max'] = (dg['circulation_temp_max'] - dg['surface_temp']) / avg_thermal_gradient

print(dg['circ_depth_best'].describe(), 
      dg['circ_depth_min'].describe(), 
      dg['circ_depth_max'].describe())

count     133.000000
mean     2111.605133
std      1126.479788
min       308.030706
25%      1170.562426
50%      1901.699175
75%      2996.819879
max      4675.836105
Name: circ_depth_best, dtype: float64 count     133.000000
mean     1644.677254
std      1155.040828
min      -223.863045
25%       685.076015
50%      1432.991204
75%      2551.144829
max      4288.658486
Name: circ_depth_min, dtype: float64 count     133.000000
mean     2430.910359
std      1007.351239
min       895.619504
25%      1594.724145
50%      2247.058172
75%      3163.441403
max      4854.533973
Name: circ_depth_max, dtype: float64


## Estimate h at max circulation depth

In [15]:
dg['h_circ_depth'] = (dg['elevation'] + dg['max_gw_lvl_elev_ws']) / 2.0

## Calculate the heat flux of each spring. 

This is the product of the difference between recharge temperature and discharge temperature of the spring, the discharge of the spring and the heat capacity and density of the spring water.

In [16]:
def calc_net_heat_flow(Q, h1, h2, T1, T2, rho=1000.0, g=9.81, c=4181.3):
    
    viscous_dissipation = rho * g * (h1 - h2) * Q
    # kg m-3 m s-2 m = kg m-1 s-2 = W
    
    H_init = rho * c * (T2 - T1) * Q
    # kg m-3 J K-1 kg-1 K m3 s-1 = J s-1 = W
    
    H = H_init - viscous_dissipation
    
    return H, H_init, viscous_dissipation


minmaxs = ['min', 'best', 'max']

for mm, mmi in zip(minmaxs, minmaxs[::-1]):

    Ts = [dg['recharge_temp_%s' % mmi], dg['circulation_temp_%s' % mm], dg['temperature']]
    hs = [dg['rch_elev_%s' % mmi], dg['h_circ_depth'], dg['elevation']]

    H_labels = ['down', 'up']

    for label, T1, T2, h1, h2 in zip(H_labels, Ts[:-1], Ts[1:], hs[:-1], hs[1:]):
        H, H_init, Hv = calc_net_heat_flow(dg['flow_rate'], h1, h2, T1, T2, rho=rho, g=g, c=c)

        dg['H_%s_%s' % (label, mm)] = H
        #dg['Hi_%s_%s' % (label, mm)] = H_init
        dg['Hv_%s_%s' % (label, mm)] = Hv
        #dg['H_div_Hv_%s_%s' % (label, mm)] = H / Hv
        
        # remove <0 heat flux estimates
        if label == 'down':
            ind_lz = dg['H_%s_%s' % (label, mm)] < 0.0
            dg.loc[ind_lz, 'H_%s_%s' % (label, mm)] = 0.0
        elif label == 'up':
            ind_lz = dg['H_%s_%s' % (label, mm)] > 0.0
            dg.loc[ind_lz, 'H_%s_%s' % (label, mm)] = 0.0
            
            
    H, Ht, Hv = calc_net_heat_flow(dg['flow_rate'], hs[0], hs[-1], Ts[0], Ts[-1], rho=rho, g=g, c=c)

    label = 'net'
    dg['H_%s_%s' % (label, mm)] = H
    #dg['Ht_%s_%s' % (label, mm)] = Ht
    dg['Hv_%s_%s' % (label, mm)] = Hv
    #dg['Hv_div_H_%s_%s' % (label, mm)] = Hv / H
    #dg['Hv_div_Ht_%s_%s' % (label, mm)] = Hv / Ht
    
    # remove <0 heat flux estimates
    ind_lz = dg['H_%s_%s' % (label, mm)] < 0.0
    dg.loc[ind_lz, 'H_%s_%s' % (label, mm)] = 0.0

In [17]:
print('spring density = 1 spring per x km = ', area_alps / 395 / 1e6)
print('=radius of ', np.sqrt(area_alps / 395 / np.pi) / 1e3)

spring density = 1 spring per x km =  512.2933983614238
=radius of  12.769810230584172


## Estimate heat flow for springs without temperature or discharge data

In [18]:
ind = dg['temperature'].notnull()
nind = dg['temperature'].isnull()
dg['temperature_est'] = dg['temperature']
dg.loc[nind, 'temperature_est'] = dg['temperature'].dropna().mean()

ind = dg['flow_rate'].notnull()
nind = dg['flow_rate'].isnull()
dg['flow_rate_est'] = dg['flow_rate']
dg.loc[nind, 'flow_rate_est'] = dg['flow_rate'].dropna().mean()

for mm, mmi in zip(minmaxs, minmaxs[::-1]):
    #ind = dg['flow_rate'].notnull()
    nind = dg['circulation_temp_%s' % mm].isnull()
    dg['circulation_temp_est_%s' % mm] = dg['circulation_temp_%s' % mm]
    dg.loc[nind, 'circulation_temp_est_%s' % mm] = dg['circulation_temp_%s' % mm].dropna().mean()

minmaxs = ['min', 'best', 'max']

for mm, mmi in zip(minmaxs, minmaxs[::-1]):

    Ts = [dg['recharge_temp_%s' % mmi], dg['circulation_temp_est_%s' % mm], dg['temperature_est']]
    hs = [dg['rch_elev_%s' % mmi], dg['h_circ_depth'], dg['elevation']]

    H_labels = ['down', 'up']

    for label, T1, T2, h1, h2 in zip(H_labels, Ts[:-1], Ts[1:], hs[:-1], hs[1:]):
        H, H_init, Hv = calc_net_heat_flow(dg['flow_rate_est'], h1, h2, T1, T2, rho=rho, g=g, c=c)

        dg['H_%s_est_%s' % (label, mm)] = H
        #dg['Hi_%s_est_%s' % (label, mm)] = H_init
        dg['Hv_%s_est_%s' % (label, mm)] = Hv
        #dg['H_div_Hv_%s_est_%s' % (label, mm)] = H / Hv

        # remove <0 heat flux estimates
        if label == 'down':
            ind_lz = dg['H_%s_est_%s' % (label, mm)] < 0.0
            dg.loc[ind_lz, 'H_%s_est_%s' % (label, mm)] = 0.0
        elif label == 'up':
            ind_lz = dg['H_%s_est_%s' % (label, mm)] > 0.0
            dg.loc[ind_lz, 'H_%s_est_%s' % (label, mm)] = 0.0
            
    H, Ht, Hv = calc_net_heat_flow(dg['flow_rate_est'], hs[0], hs[-1], Ts[0], 
                                   Ts[-1], rho=rho, g=g, c=c)

    label = 'net'
    dg['H_%s_est_%s' % (label, mm)] = H
    #dg['Ht_%s_est_%s' % (label, mm)] = Ht
    dg['Hv_%s_est_%s' % (label, mm)] = Hv
    #dg['H_div_Hv_%s_est_%s' % (label, mm)] = H / Hv
    #dg['Ht_div_Hv_%s_est_%s' % (label, mm)] = Ht / Hv
    
    # remove <0 heat flux estimates
    ind_lz = dg['H_%s_est_%s' % (label, mm)] < 0.0
    dg.loc[ind_lz, 'H_%s_est_%s' % (label, mm)] = 0.0

In [19]:
dg['temperature'].isnull().unique()

array([False,  True])

## Calculate thermal footprint

In [20]:
H_labels = ['net', 'down', 'up']

for label in H_labels:
    for mm, mmi in zip(minmaxs, minmaxs[::-1]):
        dg['thermal_footprint_%s_%s' % (label, mm)] = \
            np.abs(dg['H_%s_%s' % (label, mm)]) / background_HF
        dg['thermal_radius_%s_%s' % (label, mm)] = \
            np.sqrt(np.abs(dg['thermal_footprint_%s_%s' % (label, mm)]) / np.pi)
#
for label in H_labels:
    print('\nthermal footprint (m2)%s:\n' % label, dg['thermal_footprint_%s_best' % label].describe())
    print('\nthermal footprint (m2)%s:\n' % label, dg['thermal_footprint_%s_min' % label].describe())
    print('\nthermal footprint (m2)%s:\n' % label, dg['thermal_footprint_%s_max' % label].describe())
    print('\nthermal radius (m)%s:\n' % label, dg['thermal_radius_%s_best' % label].describe())
    print('\nthermal radius (m)%s:\n' % label, dg['thermal_radius_%s_min' % label].describe())
    print('\nthermal radius (m)%s:\n' % label, dg['thermal_radius_%s_max' % label].describe())


thermal footprint (m2)net:
 count    2.260000e+02
mean     6.503491e+06
std      1.518527e+07
min      0.000000e+00
25%      1.253791e+05
50%      9.421131e+05
75%      4.480715e+06
max      1.196846e+08
Name: thermal_footprint_net_best, dtype: float64

thermal footprint (m2)net:
 count    2.260000e+02
mean     4.918801e+06
std      1.208200e+07
min      0.000000e+00
25%      0.000000e+00
50%      3.114644e+05
75%      3.282303e+06
max      9.290677e+07
Name: thermal_footprint_net_min, dtype: float64

thermal footprint (m2)net:
 count    2.260000e+02
mean     8.795423e+06
std      1.916466e+07
min      1.467466e+03
25%      2.350240e+05
50%      1.493743e+06
75%      6.983339e+06
max      1.464624e+08
Name: thermal_footprint_net_max, dtype: float64

thermal radius (m)net:
 count     226.000000
mean      932.696252
std      1097.969662
min         0.000000
25%       199.773031
50%       547.612122
75%      1194.147747
max      6172.259530
Name: thermal_radius_net_best, dtype: float64



## Calculation example

In [21]:
median_spring_temp = Ts[-1].median()
median_rch_temp = Ts[0].median()
median_spring_elev = hs[-1].median()
median_rch_elev = hs[0].median()
median_Q = dg['flow_rate'].median()

print('Q, rch T, spring T, rch z, spring T for median spring: ', 
      median_Q, median_rch_temp, median_spring_temp, median_rch_elev, median_spring_elev)

Q, rch T, spring T, rch z, spring T for median spring:  0.0017 1.2166666801397998 21.15 651.0 651.0


## Fill in the heat flux of springs with no T or Q data

not needed anymore, estimated values of T, Q, circ. temp in one of the code blocks above

In [22]:
print('net heat flux:\n', dg['H_net_best'].describe())
print('net heat flux:\n', dg['H_net_min'].describe())
print('net heat flux:\n', dg['H_net_max'].describe())

print('\nnet heat flux estimated:\n', dg['H_net_est_best'].describe())

net heat flux:
 count    2.260000e+02
mean     4.942653e+05
std      1.154080e+06
min      0.000000e+00
25%      9.528812e+03
50%      7.160059e+04
75%      3.405343e+05
max      9.096029e+06
Name: H_net_best, dtype: float64
net heat flux:
 count    2.260000e+02
mean     3.738289e+05
std      9.182320e+05
min      0.000000e+00
25%      0.000000e+00
50%      2.367129e+04
75%      2.494550e+05
max      7.060914e+06
Name: H_net_min, dtype: float64
net heat flux:
 count    2.260000e+02
mean     6.684521e+05
std      1.456514e+06
min      1.115274e+02
25%      1.786182e+04
50%      1.135245e+05
75%      5.307338e+05
max      1.113114e+07
Name: H_net_max, dtype: float64

net heat flux estimated:
 count    3.940000e+02
mean     5.818037e+05
std      1.224492e+06
min      0.000000e+00
25%      1.500671e+04
50%      1.622085e+05
75%      7.206904e+05
max      1.415105e+07
Name: H_net_est_best, dtype: float64


## Save the modified csv file with the additional heat flux data

In [23]:
spring_data_file_mod = spring_data_file.split('with')[0] + 'with_HF_estimates.csv'

print('saving modified csv file as ', spring_data_file_mod)
dg.to_csv(spring_data_file_mod, index=False, index_label=False, encoding=encoding)

print('done')

saving modified csv file as  data/thermal_springs_alps_with_HF_estimates.csv
done


## Save clean version with the estimate HF values omitted

In [24]:
spring_data_file_clean = spring_data_file.split('with')[0] + 'with_HF_estimates_clean.csv'

cols_to_save = dg.columns

cols_to_save = [c for c in cols_to_save if '_est' not in c]

cols_to_save.remove('3He')
cols_to_save.remove('4He')

dc = dg[cols_to_save]

chem_cols = ['TDS', 'TDS_min', 'TDS_max', 'Na', 'Ca', 'Mg', 'K', 
             'NH4', 'Cl', 'F', 'SO4', 'HCO3', 'CO3', 'NO3', 
             'Si', 'Li', 'SiO2']

# rename columsn to add units
for c in cols_to_save:
    nc = c
    if 'circ_depth' in c:
        nc += '_(m)'
    elif c[:2] == 'H_':
        nc += '_(W)'
    elif c[:3] == 'Hv_':
        nc += '_(W)'
    elif 'depth' in c:
        nc += '_(m)'
    elif 'temp' in c:
        nc += '_(degr_C)'
    elif 'footprint' in c:
        nc += '_(m^2)'
    elif 'radius' in c:
        nc += '_(m)'
    elif 'elev' in c:
        nc += '_(m)'
    elif 'relief' in c:
        nc += '_(m)'
    elif 'gw_lvl' in c:
        nc += '_(m)'
    elif 'area' in c:
        nc += '_(m^2)'
    elif 'flow_rate' in c:
        nc += '_(m^3_s^-1)'
    elif 'recharge' in c:
        nc += '_(m_a^-1)'
    elif c in chem_cols:
        nc += '_(mg L^-1)'
    elif c == '3H':
        nc += '_(TU)'
    elif 'delta_14C' in c:
        nc += '_(pmc)'
    elif 'delta' in c:
        nc += '_(permille)'
    elif 'EC' in c:
        nc += '_(S_m^-1)'
    elif 'long' in c:
        nc = 'longitude'
    elif 'lat' in c:
        nc = 'latitude'
    
    dc = dc.rename(columns={c:nc})  

# sort HF, thermal footprint and thermal radius cols
c = dc.columns
reorg_cols = ['H_net', 'H_down', 'H_up', 'Hv_net', 'thermal_footprint', 'thermal_radius']

nc = c
for ri in reorg_cols:
    nc = [ci for ci in nc if ri not in ci]

nc2 = nc
mm = ['best', 'min', 'max']
for r in reorg_cols:
    for mi in mm:
        c1 = [ci for ci in c if r in ci and mi in ci]
        if len(c1) > 0:
            nc2 += [c1[0]]
        
dc = dc[nc2]

# save csv file
#print('saving cleaned csv file with columns ', dc.columns.tolist())

#print('saving modified csv file as ', spring_data_file_clean)
dc.to_csv(spring_data_file_clean, index=False, index_label=False, encoding=encoding)



## Summarize results

### Create dataframe to store summary results

In [25]:
#cols = ['n_springs_total', 'n_springs_with_HF_data', 
#        'spring_density', 
#        'total_HF_min', 'total_HF_max', 
#        'total_HF_min_est', 'total_HF_max_est', 
#        'circulation_temp_min', 'circulation_temp_max',
#        'HF_up_min', 'HF_up_max', 'HF_down_min', 'HF_down_max']

cols = []

ind = ['Alps']

dfr = pd.DataFrame(index=ind, columns=cols)
#dfr.head()

## Calculate total heat flux

In [26]:

ix = 'Alps'
H_labels = ['net', 'down', 'up']

dfr.loc[ix, 'total_background_HF_MW'] = background_HF * area_alps / 1e6


for mm in minmaxs:
    for l in H_labels:
        dfr.loc[ix, 'total_H_%s_%s_MW' % (l, mm)] = dg['H_%s_%s' % (l, mm)].sum() / 1e6
        
        dfr.loc[ix, 'total_H_%s_%s_mW_m-2' % (l, mm)] = \
            dfr.loc[ix, 'total_H_%s_%s_MW' % (l, mm)]  * 1e6 / area_alps * 1000.0
        
        dfr.loc[ix, 'total_H_%s_%s_as_perc_of_background_HF' % (l, mm)] = \
            dfr.loc[ix, 'total_H_%s_%s_MW' % (l, mm)] / dfr.loc[ix, 'total_background_HF_MW'] * 100.0
        
        dfr.loc[ix, 'total_H_%s_est_%s_MW' % (l, mm)] = dg['H_%s_est_%s' % (l, mm)].sum() / 1e6
        
        dfr.loc[ix, 'total_H_%s_est_%s_mW_m-2' % (l, mm)] = \
            dfr.loc[ix, 'total_H_%s_est_%s_MW' % (l, mm)]  * 1e6 / area_alps * 1000.0
        
        dfr.loc[ix, 'total_H_%s_est_%s_as_perc_of_background_HF' % (l, mm)] = \
            dfr.loc[ix, 'total_H_%s_est_%s_MW' % (l, mm)] / dfr.loc[ix, 'total_background_HF_MW'] * 100.0

        dfr.loc[ix, 'n_springs_with_H_%s_data' % l] = len(dg['H_%s_%s' % (l, mm)].dropna())
        
        dfr.loc[ix, 'spring_density_%s_per_km2' % l] = \
            dfr.loc[ix, 'n_springs_with_H_%s_data' % l] / (area_alps / 1e6)

for l in H_labels:
    
    print('\ntotal %s heat flux of %i of a total of %i thermal springs in the Alps = %0.0f - %0.0f MW' 
          % (l, 
             dfr.loc[ix, 'n_springs_with_H_%s_data' % l] , 
             len(df),dfr.loc[ix, 'total_H_%s_min_MW' % l], 
             dfr.loc[ix, 'total_H_%s_max_MW' % l]))

    print('\twhich equals a heat flow density of %0.3e - %0.3e mW m-2' 
          % (dfr.loc[ix, 'total_H_%s_min_MW' % l] * 1e6 / area_alps * 1e3,
             dfr.loc[ix, 'total_H_%s_max_MW' % l] * 1e6 / area_alps * 1e3))
    
    print('\twhich equals %0.3f - %0.3f perc of the background heat flux' 
          % (dfr.loc[ix, 'total_H_%s_min_as_perc_of_background_HF' % l],
             dfr.loc[ix, 'total_H_%s_max_as_perc_of_background_HF' % l]))

    print('\ntotal estimated %s heat flux of %i of a total of %i thermal springs in the Alps = %0.0f - %0.0f MW' 
          % (l, 
             len(dg) , 
             len(df),
             dfr.loc[ix, 'total_H_%s_est_min_MW' % l], 
             dfr.loc[ix, 'total_H_%s_est_max_MW' % l]))
    
    print('\twhich equals a heat flow density of %0.3e - %0.3e mW m-2' 
          % (dfr.loc[ix, 'total_H_%s_est_min_MW' % l] * 1e6 / area_alps * 1e3,
             dfr.loc[ix, 'total_H_%s_est_max_MW' % l] * 1e6 / area_alps * 1e3))
    
    print('\twhich equals %0.3f - %0.3f perc of the background heat flux' 
          % (dfr.loc[ix, 'total_H_%s_est_min_as_perc_of_background_HF' % l],
             dfr.loc[ix, 'total_H_%s_est_max_as_perc_of_background_HF' % l]))


total net heat flux of 226 of a total of 394 thermal springs in the Alps = 84 - 151 MW
	which equals a heat flow density of 4.175e-01 - 7.466e-01 mW m-2
	which equals 0.549 - 0.982 perc of the background heat flux

total estimated net heat flux of 394 of a total of 394 thermal springs in the Alps = 172 - 295 MW
	which equals a heat flow density of 8.485e-01 - 1.457e+00 mW m-2
	which equals 1.117 - 1.917 perc of the background heat flux

total down heat flux of 64 of a total of 394 thermal springs in the Alps = 55 - 97 MW
	which equals a heat flow density of 2.717e-01 - 4.812e-01 mW m-2
	which equals 0.357 - 0.633 perc of the background heat flux

total estimated down heat flux of 394 of a total of 394 thermal springs in the Alps = 741 - 1266 MW
	which equals a heat flow density of 3.661e+00 - 6.256e+00 mW m-2
	which equals 4.817 - 8.231 perc of the background heat flux

total up heat flux of 62 of a total of 394 thermal springs in the Alps = -35 - -65 MW
	which equals a heat flow dens

## Compare thermal springs flux to total groundwater flux

In [27]:
total_rch = area_alps * dg['recharge'].mean()

year = 365.25 * 24 * 3600.
print('total recharge = %0.2e km3 a-1' % (total_rch / 1e9))

print('contribution to groundwater budget of n=%i springs = %0.2e m3 s-1 = %0.2e mm a-1 = %0.2e percent of total recharge in Alps' 
      % (dg['flow_rate'].notnull().sum(), 
         dg['flow_rate'].sum(),
         dg['flow_rate'].sum() * year / area_alps * 1e3,
         dg['flow_rate'].sum() * year / total_rch * 100))

dg['flow_rate_est'] = dg['flow_rate']
ok = dg['flow_rate'].notnull()
nok = dg['flow_rate'].isnull()
dfr.loc[ix, 'mean_flow_rate'] = dg.loc[ok, 'flow_rate'].mean()
dg.loc[nok, 'flow_rate_est'] = dfr.loc[ix, 'mean_flow_rate']

dfr.loc[ix, 'n_springs'] = dg['flow_rate_est'].notnull().sum()
dfr.loc[ix, 'area_km2'] = area_alps / 1e6
dfr.loc[ix, 'median_temperature'] = np.median(dg['temperature'].dropna())
dfr.loc[ix, 'total_spring_flux_km3_yr-1'] = (dg['flow_rate'].sum() * year / 1e9)
dfr.loc[ix, 'total_spring_flux_est_km3_yr-1'] = (dg['flow_rate_est'].sum() * year / 1e9)
dfr.loc[ix, 'total_recharge_km3_yr-1'] =  total_rch / 1e9
dfr.loc[ix, 'percentage_of_total_meteoric_gw']  = dg['flow_rate_est'].sum() * year / total_rch * 100.0

print('estimated flow rate for springs with no flow rate data is equal to the mean flow rate of %0.2e' % dfr.loc[ix, 'mean_flow_rate'] )
print('mean recharge rate  m / yr = %0.2e' % dg['recharge'].mean())
print('total recharge rate = %0.2e km3 / year' % (total_rch/1e9))
print('estimated total spring water flux = %0.2e m3 s-1 = %0.2e km3/yr = %0.2e mm' % (dg['flow_rate_est'].sum(), 
                                                                                      dfr.loc[ix, 'total_spring_flux_est_km3_yr-1'],
                                                                                      dg['flow_rate_est'].sum() * year / area_alps * 1e3))
print('estimated contribution to meteoric groundwater budget of all n=%i springs = %0.2e percent ' 
      % (dg['flow_rate_est'].notnull().sum(), dfr.loc[ix, 'percentage_of_total_meteoric_gw']))

total recharge = 1.03e+02 km3 a-1
contribution to groundwater budget of n=241 springs = 2.67e+00 m3 s-1 = 4.16e-01 mm a-1 = 8.17e-02 percent of total recharge in Alps
estimated flow rate for springs with no flow rate data is equal to the mean flow rate of 1.11e-02
mean recharge rate  m / yr = 5.09e-01
total recharge rate = 1.03e+02 km3 / year
estimated total spring water flux = 4.36e+00 m3 s-1 = 1.38e-01 km3/yr = 6.81e-01 mm
estimated contribution to meteoric groundwater budget of all n=394 springs = 1.34e-01 percent 


## some more stats

In [28]:
print('mean discharge for %i springs = %0.2e m3/sec' % (dg['flow_rate'].notnull().sum(),
                                                        dg['flow_rate'].mean()))

print('mean net heat flux = %0.2e (%0.2e - %0.2e) W' % (dg['H_net_best'].mean(),
                                                       dg['H_net_min'].mean(),
                                                       dg['H_net_max'].mean()))

print('mean thermal footprint (km2) = %0.2e (%0.2e - %0.2e)' 
      % (df['thermal_footprint_net_best'].mean(), 
         df['thermal_footprint_net_min'].mean(), 
         df['thermal_footprint_net_max'].mean()))
print('mean thermal radius (m) = %0.2e (%0.2e - %0.2e)' 
      %(df['thermal_radius_net_best'].mean(), 
        df['thermal_radius_net_min'].mean(), 
        df['thermal_radius_net_max'].mean()))

mean discharge for 241 springs = 1.11e-02 m3/sec
mean net heat flux = 4.94e+05 (3.74e+05 - 6.68e+05) W
mean thermal footprint (km2) = 6.50e+06 (4.92e+06 - 8.80e+06)
mean thermal radius (m) = 9.33e+02 (7.24e+02 - 1.13e+03)


## Save summary csv file

In [29]:
summary_file = 'data/summary_thermal_springs_data.csv'
print('saving summary data file as %s' % summary_file)
dfr.to_csv(summary_file, float_format='%.2f', index_label='dataset')

saving summary data file as data/summary_thermal_springs_data.csv
